In [1]:
import numpy as np
from pathlib import Path

In [2]:
tile_dir = Path("../../data/npz_all/npz/tile/xla/")
train_dir = tile_dir / "train"
valid_dir = tile_dir / "valid"
test_dir = tile_dir / "test"


In [15]:
all_files = list(train_dir.glob("*.npz")) + list(valid_dir.glob("*.npz")) + list(test_dir.glob("*.npz"))

In [6]:
for file in train_dir.iterdir():
    print(file)
    data = np.load(file)
    print(data)
    break

../../data/npz_all/npz/tile/xla/train/mlperf_maskrcnn_batch_2_-18c2c44102113665.npz


In [8]:
data.files

['node_feat',
 'node_opcode',
 'edge_index',
 'config_feat',
 'config_runtime',
 'config_runtime_normalizers']

In [9]:
feats= ['node_feat',
 'node_opcode',
 'edge_index',
 'config_feat',
 'config_runtime',
 'config_runtime_normalizers']

for f in feats:
    print(f, data[f].shape)

node_feat (20, 140)
node_opcode (20,)
edge_index (19, 2)
config_feat (8209, 24)
config_runtime (8209,)
config_runtime_normalizers (8209,)


In [18]:
tile_norms = {}

In [19]:
node_feats = []
for file in all_files:
    npz = np.load(file)
    # config_feat = npz["config_feat"]
    node_feat = npz["node_feat"]
    node_feats.append(node_feat)
    
node_feats = np.concatenate(node_feats, axis=0)
# data["config_feat"].min(axis=0) == data["config_feat"].max(axis=0)

tile_norms["node_feat_mask"] = (node_feats.min(axis=0) != node_feats.max(axis=0)).tolist()
tile_norms["node_feat_min"] = node_feats.min(axis=0).tolist()
tile_norms["node_feat_max"] = node_feats.max(axis=0).tolist()


In [28]:
config_feats = []
for file in all_files:
    npz = np.load(file)
    config_feat = npz["config_feat"]
    config_feats.append(config_feat)
    
config_feats = np.concatenate(config_feats, axis=0)

tile_norms["config_feat_mask"] = (config_feats.min(axis=0) != config_feats.max(axis=0)).tolist()
tile_norms["config_feat_min"] = config_feats.min(axis=0).tolist()
tile_norms["config_feat_max"] = config_feats.max(axis=0).tolist()


In [32]:
import json

In [33]:
json.dump(tile_norms, open("../../data/tile_normalizers.json", "w"))